# **<span style="color:#4682B4"><u>Model Training and Evaluation</u></span>**

---
## **<span style="color:#483D8B"><u>Workflow</u></span>**
- Import Libraries
- Load Preprocessed Data
- Define Evaluation Metrics
- Model Selection and Training
- Model Comparison
- Save Pre-Trained Models
- Test Data Evaluation
- Final Summary

---
### **<span style="color:#8B008B"><u>Import Libraries</u></span>**

In [1]:
# Core Libraries
import pandas as pd
import numpy as np
import json
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# Fix macOS MallocStackLogging warning with XGBoost/OpenMP
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OBJC_DISABLE_INITIALIZE_FORK_SAFETY'] = 'YES'

# Visualization - Matplotlib & Seaborn
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization - Plotly (for interactive HTML plots)
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Sklearn - Metrics
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    log_loss
)
from sklearn.preprocessing import label_binarize

# Sklearn - Models
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

print("All libraries imported successfully!")

All libraries imported successfully!


---
### **<span style="color:#8B008B"><u>Load Preprocessed Data</u></span>**

In [2]:
# Configuration - File Paths
ARTIFACTS_DIR = '../artifacts'
DATA_DIR = f'{ARTIFACTS_DIR}/data'
REPORTS_DIR = f'{ARTIFACTS_DIR}/reports'
MODELS_DIR = '../models'
DATASET_METADATA_PATH = '../dataset/dataset.json'

# Load dataset metadata to get dataset name and target
with open(DATASET_METADATA_PATH, 'r') as f:
    dataset_metadata = json.load(f)

# Get dataset display name from metadata
DATASET_DISPLAY_NAME = dataset_metadata['dataset_name']

# Get short_name from metadata for file naming convention (fallback to cleaned dataset_name)
DATASET_NAME = dataset_metadata.get('short_name', DATASET_DISPLAY_NAME.lower().replace(' ', '_').replace('-', '_'))

# Get target column name from metadata
TARGET_COLUMN_ORIGINAL = dataset_metadata['target']  # Original target name (e.g., 'NObeyesdad')
TARGET_COLUMN = 'target'  # Preprocessed target column name (encoded)

# Derived file paths
PREPROCESSING_ANALYSIS_PATH = f'{DATA_DIR}/{DATASET_NAME}_preprocessing_analysis.json'
TRAIN_DATA_PATH = f'{DATA_DIR}/{DATASET_NAME}_train_preprocessed.csv'
VALIDATION_DATA_PATH = f'{DATA_DIR}/{DATASET_NAME}_validation_preprocessed.csv'

print("Configuration:")
print(f"   Dataset Name: {DATASET_DISPLAY_NAME}")
print(f"   Dataset Short Name (for files): {DATASET_NAME}")
print(f"   Target Column (Original): {TARGET_COLUMN_ORIGINAL}")
print(f"   Target Column (Preprocessed): {TARGET_COLUMN}")
print(f"   Preprocessing Analysis: {PREPROCESSING_ANALYSIS_PATH}")
print(f"   Training Data: {TRAIN_DATA_PATH}")
print(f"   Validation Data: {VALIDATION_DATA_PATH}")
print(f"   Models Directory: {MODELS_DIR}")
print(f"   Reports Directory: {REPORTS_DIR}")

Configuration:
   Dataset Name: Multi-Class Prediction of Obesity Risk
   Dataset Short Name (for files): kaggle_obesity_prediction
   Target Column (Original): NObeyesdad
   Target Column (Preprocessed): target
   Preprocessing Analysis: ../artifacts/data/kaggle_obesity_prediction_preprocessing_analysis.json
   Training Data: ../artifacts/data/kaggle_obesity_prediction_train_preprocessed.csv
   Validation Data: ../artifacts/data/kaggle_obesity_prediction_validation_preprocessed.csv
   Models Directory: ../models
   Reports Directory: ../artifacts/reports


In [3]:
# Load Preprocessing Analysis Configuration
with open(PREPROCESSING_ANALYSIS_PATH, 'r') as f:
    preprocessing_analysis = json.load(f)

# Extract configuration from preprocessing analysis
feature_names = preprocessing_analysis['feature_names']
target_encoding = preprocessing_analysis['preprocessing_stats']['target_encoding']
class_names = target_encoding['encoder_classes']
num_classes = target_encoding['num_classes']
class_to_label = target_encoding['class_to_label']
label_to_class = target_encoding['label_to_class']

print("Preprocessing analysis loaded successfully!")
print(f"\nDataset Configuration:")
print(f"   Train Shape: {preprocessing_analysis['train_shape']}")
print(f"   Validation Shape: {preprocessing_analysis['validation_shape']}")
print(f"   Number of Features: {len(feature_names)}")
print(f"   Number of Classes: {num_classes}")
print(f"   Class Names: {class_names}")

Preprocessing analysis loaded successfully!

Dataset Configuration:
   Train Shape: [15879, 23]
   Validation Shape: [2803, 23]
   Number of Features: 23
   Number of Classes: 7
   Class Names: ['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I', 'Overweight_Level_II']


In [4]:
# Load Preprocessed Data
train_df = pd.read_csv(TRAIN_DATA_PATH)
val_df = pd.read_csv(VALIDATION_DATA_PATH)

print(f"Data loaded successfully!")
print(f"   Training data shape: {train_df.shape}")
print(f"   Validation data shape: {val_df.shape}")

Data loaded successfully!
   Training data shape: (15879, 24)
   Validation data shape: (2803, 24)


In [5]:
# Prepare Features and Target
# Exclude 'id' and target columns from features
feature_columns = [col for col in train_df.columns if col not in ['id', TARGET_COLUMN]]

X_train = train_df[feature_columns]
y_train = train_df[TARGET_COLUMN]

X_val = val_df[feature_columns]
y_val = val_df[TARGET_COLUMN]

print(f"Features and targets prepared!")
print(f"   Target Column: {TARGET_COLUMN} (originally: {TARGET_COLUMN_ORIGINAL})")
print(f"   Training features shape: {X_train.shape}")
print(f"   Training target shape: {y_train.shape}")
print(f"   Validation features shape: {X_val.shape}")
print(f"   Validation target shape: {y_val.shape}")
print(f"\nTarget distribution in training set:")
print(y_train.value_counts().sort_index())

Features and targets prepared!
   Target Column: target (originally: NObeyesdad)
   Training features shape: (15879, 22)
   Training target shape: (15879,)
   Validation features shape: (2803, 22)
   Validation target shape: (2803,)

Target distribution in training set:
target
0    1930
1    2358
2    2226
3    2485
4    3095
5    1856
6    1929
Name: count, dtype: int64


---
### **<span style="color:#8B008B"><u>Define Evaluation Metrics</u></span>**

In [6]:
def calculate_metrics(y_true, y_pred, y_pred_proba=None, num_classes=7):
    """
    Calculate all evaluation metrics for multi-class classification.
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    y_pred_proba : array-like, optional
        Predicted probabilities for AUC calculation
    num_classes : int
        Number of classes
        
    Returns:
    --------
    dict : Dictionary containing all metrics
    """
    metrics = {}
    
    # 1. Accuracy
    metrics['Accuracy'] = accuracy_score(y_true, y_pred)
    
    # 2. AUC Score (One-vs-Rest for multi-class)
    if y_pred_proba is not None:
        try:
            metrics['AUC_Score'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['AUC_Score'] = None
    else:
        metrics['AUC_Score'] = None
    
    # 3. Precision (weighted average for multi-class)
    metrics['Precision'] = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # 4. Recall (weighted average for multi-class)
    metrics['Recall'] = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # 5. F1 Score (weighted average for multi-class)
    metrics['F1_Score'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # 6. Matthews Correlation Coefficient
    metrics['MCC_Score'] = matthews_corrcoef(y_true, y_pred)
    
    return metrics


def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    """
    Train and evaluate a model, returning comprehensive metrics.
    
    Parameters:
    -----------
    model : sklearn estimator
        The model to train
    X_train, y_train : training data
    X_val, y_val : validation data
    model_name : str
        Name of the model for display
        
    Returns:
    --------
    dict : Dictionary containing training and validation metrics
    """
    print(f"\n{'='*100}")
    print(f"Training: {model_name}")
    print(f"{'='*100}")
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predictions for training set
    y_train_pred = model.predict(X_train)
    y_train_proba = model.predict_proba(X_train) if hasattr(model, 'predict_proba') else None
    
    # Predictions for validation set
    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val) if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    val_metrics = calculate_metrics(y_val, y_val_pred, y_val_proba)
    
    # Display results
    print(f"\nTraining Metrics:")
    for metric, value in train_metrics.items():
        if value is not None:
            print(f"   {metric}: {value:.4f}")
    
    print(f"\nValidation Metrics:")
    for metric, value in val_metrics.items():
        if value is not None:
            print(f"   {metric}: {value:.4f}")
    
    return {
        'model_name': model_name,
        'model': model,
        'train_metrics': train_metrics,
        'val_metrics': val_metrics,
        'y_val_pred': y_val_pred,
        'y_val_proba': y_val_proba
    }

print("Evaluation metrics function defined successfully!")

Evaluation metrics function defined successfully!


---
### **<span style="color:#8B008B"><u>Model Selection and Training</u></span>**
#### **<span style="color:#8B4513"><u>HyperParameter Search Space</u></span>**

In [7]:
# Define hyperparameter search spaces for each model
param_distributions = {
    'Logistic Regression': {
        'C': uniform(0.01, 10),  # Regularization strength
        'penalty': ['l2'],
        'solver': ['lbfgs', 'saga'],
        'max_iter': [200]
    },
    'Decision Tree Classifier': {
        'max_depth': randint(5, 30),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10),
        'criterion': ['gini', 'entropy'],
        'max_features': ['sqrt', 'log2', None]
    },
    'K-Nearest Neighbor Classifier': {
        'n_neighbors': randint(3, 25),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'p': [1, 2, 3]
    },
    'Random Forest (Ensemble)': {
        'n_estimators': randint(100, 500),
        'max_depth': randint(10, 30),
        'min_samples_split': randint(2, 15),
        'min_samples_leaf': randint(1, 8),
        'max_features': ['sqrt', 'log2'],
        'bootstrap': [True],
        'class_weight': ['balanced', 'balanced_subsample', None]
    },
    'XGBoost (Ensemble)': {
        'n_estimators': randint(100, 400),
        'max_depth': randint(4, 12),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'reg_alpha': uniform(0, 1),  # L1 regularization
        'reg_lambda': uniform(0, 2),  # L2 regularization
        'min_child_weight': randint(1, 10)
    }
}

print("Hyperparameter search spaces defined for tuning!")

Hyperparameter search spaces defined for tuning!


#### **<span style="color:#8B4513"><u>HyperParameter Tuning</u></span>**

In [8]:
# Define all 6 models with default parameters for tuning
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree Classifier': DecisionTreeClassifier(random_state=42),
    'K-Nearest Neighbor Classifier': KNeighborsClassifier(),
    'Naive Bayes Classifier (Gaussian)': GaussianNB(),
    'Random Forest (Ensemble)': RandomForestClassifier(random_state=42, n_jobs=-1),
    'XGBoost (Ensemble)': XGBClassifier(
        objective='multi:softprob',
        num_class=num_classes,
        random_state=42,
        eval_metric='mlogloss',
        n_jobs=-1,
        verbosity=0
    )
}

# Define hyperparameter search spaces for each model
param_distributions = {
    'Logistic Regression': {
        'C': uniform(0.01, 10),
        'penalty': ['l2'],
        'solver': ['lbfgs', 'saga'],
        'max_iter': [2000]
    },
    'Decision Tree Classifier': {
        'max_depth': randint(5, 30),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10),
        'criterion': ['gini', 'entropy'],
        'max_features': ['sqrt', 'log2', None]
    },
    'K-Nearest Neighbor Classifier': {
        'n_neighbors': randint(3, 25),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'p': [1, 2, 3]
    },
    'Naive Bayes Classifier (Gaussian)': {
        'var_smoothing': uniform(1e-10, 1e-8)
    },
    'Random Forest (Ensemble)': {
        'n_estimators': randint(100, 500),
        'max_depth': randint(10, 30),
        'min_samples_split': randint(2, 15),
        'min_samples_leaf': randint(1, 8),
        'max_features': ['sqrt', 'log2'],
        'bootstrap': [True],
        'class_weight': ['balanced', 'balanced_subsample', None]
    },
    'XGBoost (Ensemble)': {
        'n_estimators': randint(100, 400),
        'max_depth': randint(4, 12),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'reg_alpha': uniform(0, 1),
        'reg_lambda': uniform(0, 2),
        'min_child_weight': randint(1, 10)
    }
}

print(f"{len(models)} models defined for hyperparameter tuning:")
for i, name in enumerate(models.keys(), 1):
    print(f"   {i}. {name}")

print("\nStarting Hyperparameter Tuning with RandomizedSearchCV (5-fold CV)...")
print("=" * 100)

# Store tuning results
tuning_results = {}
tuned_models = {}

for model_name, model in models.items():
    print(f"\nTuning: {model_name}")
    
    # RandomizedSearchCV with 5-fold cross-validation
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions[model_name],
        n_iter=30,
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    # Fit on training data
    search.fit(X_train, y_train)
    
    # Store best model and results
    tuned_models[model_name] = search.best_estimator_
    tuning_results[model_name] = {
        'best_params': search.best_params_,
        'best_cv_score': search.best_score_,
        'cv_std': search.cv_results_['std_test_score'][search.best_index_]
    }
    
    print(f"   Best CV Score: {search.best_score_:.4f} (+/- {tuning_results[model_name]['cv_std']:.4f})")
    print(f"   Best Parameters: {search.best_params_}")

print("\n" + "=" * 100)
print("Hyperparameter tuning completed!")

# Use tuned models for evaluation
models = tuned_models

6 models defined for hyperparameter tuning:
   1. Logistic Regression
   2. Decision Tree Classifier
   3. K-Nearest Neighbor Classifier
   4. Naive Bayes Classifier (Gaussian)
   5. Random Forest (Ensemble)
   6. XGBoost (Ensemble)

Starting Hyperparameter Tuning with RandomizedSearchCV (5-fold CV)...

Tuning: Logistic Regression
   Best CV Score: 0.8640 (+/- 0.0018)
   Best Parameters: {'C': np.float64(9.74755518841459), 'max_iter': 2000, 'penalty': 'l2', 'solver': 'lbfgs'}

Tuning: Decision Tree Classifier
   Best CV Score: 0.8739 (+/- 0.0042)
   Best Parameters: {'criterion': 'gini', 'max_depth': 9, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2}

Tuning: K-Nearest Neighbor Classifier
   Best CV Score: 0.7819 (+/- 0.0070)
   Best Parameters: {'metric': 'minkowski', 'n_neighbors': 11, 'p': 1, 'weights': 'distance'}

Tuning: Naive Bayes Classifier (Gaussian)
   Best CV Score: 0.6097 (+/- 0.0127)
   Best Parameters: {'var_smoothing': np.float64(9.607143064099161e-

#### **<span style="color:#8B4513"><u>Model Training</u></span>**

In [9]:
# Train and evaluate all models
results = {}

for model_name, model in models.items():
    results[model_name] = evaluate_model(
        model, X_train, y_train, X_val, y_val, model_name
    )

print(f"\n{'='*100}")
print("All models trained and evaluated successfully!")
print(f"{'='*100}")


Training: Logistic Regression

Training Metrics:
   Accuracy: 0.8672
   AUC_Score: 0.9828
   Precision: 0.8659
   Recall: 0.8672
   F1_Score: 0.8663
   MCC_Score: 0.8444

Validation Metrics:
   Accuracy: 0.8644
   AUC_Score: 0.9818
   Precision: 0.8638
   Recall: 0.8644
   F1_Score: 0.8638
   MCC_Score: 0.8411

Training: Decision Tree Classifier

Training Metrics:
   Accuracy: 0.9026
   AUC_Score: 0.9888
   Precision: 0.9029
   Recall: 0.9026
   F1_Score: 0.9026
   MCC_Score: 0.8858

Validation Metrics:
   Accuracy: 0.8776
   AUC_Score: 0.9749
   Precision: 0.8787
   Recall: 0.8776
   F1_Score: 0.8779
   MCC_Score: 0.8566

Training: K-Nearest Neighbor Classifier

Training Metrics:
   Accuracy: 1.0000
   AUC_Score: 1.0000
   Precision: 1.0000
   Recall: 1.0000
   F1_Score: 1.0000
   MCC_Score: 1.0000

Validation Metrics:
   Accuracy: 0.7752
   AUC_Score: 0.9501
   Precision: 0.7736
   Recall: 0.7752
   F1_Score: 0.7731
   MCC_Score: 0.7366

Training: Naive Bayes Classifier (Gaussian)



---
### **<span style="color:#8B008B"><u>Model Comparison</u></span>**

In [10]:
# Create comparison DataFrames for Training and Validation metrics
train_comparison = []
val_comparison = []

for model_name, result in results.items():
    train_row = {'Model': model_name}
    train_row.update(result['train_metrics'])
    train_comparison.append(train_row)
    
    val_row = {'Model': model_name}
    val_row.update(result['val_metrics'])
    val_comparison.append(val_row)

train_comparison_df = pd.DataFrame(train_comparison)
val_comparison_df = pd.DataFrame(val_comparison)

# Display Training Metrics Comparison
print("Training Metrics Comparison")
print("="*100)
display(train_comparison_df.style.format({
    'Accuracy': '{:.4f}',
    'AUC_Score': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1_Score': '{:.4f}',
    'MCC_Score': '{:.4f}'
}).highlight_max(subset=['Accuracy', 'AUC_Score', 'Precision', 'Recall', 'F1_Score', 'MCC_Score'],  color='darkgreen'))

Training Metrics Comparison


,Model,Accuracy,AUC_Score,Precision,Recall,F1_Score,MCC_Score
0,Logistic Regression,0.8672,0.9828,0.8659,0.8672,0.8663,0.8444
1,Decision Tree Classifier,0.9026,0.9888,0.9029,0.9026,0.9026,0.8858
2,K-Nearest Neighbor Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,Naive Bayes Classifier (Gaussian),0.6159,0.9160,0.6243,0.6159,0.5844,0.5584
4,Random Forest (Ensemble),0.9673,0.9991,0.9673,0.9673,0.9672,0.9617
5,XGBoost (Ensemble),0.9735,0.9992,0.9734,0.9735,0.9734,0.9689


In [11]:
# Display Validation Metrics Comparison
print("\nValidation Metrics Comparison")
print("="*100)
display(val_comparison_df.style.format({
    'Accuracy': '{:.4f}',
    'AUC_Score': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1_Score': '{:.4f}',
    'MCC_Score': '{:.4f}'
}).highlight_max(subset=['Accuracy', 'AUC_Score', 'Precision', 'Recall', 'F1_Score', 'MCC_Score'],  color='darkgreen'))


Validation Metrics Comparison


,Model,Accuracy,AUC_Score,Precision,Recall,F1_Score,MCC_Score
0,Logistic Regression,0.8644,0.9818,0.8638,0.8644,0.8638,0.8411
1,Decision Tree Classifier,0.8776,0.9749,0.8787,0.8776,0.8779,0.8566
2,K-Nearest Neighbor Classifier,0.7752,0.9501,0.7736,0.7752,0.7731,0.7366
3,Naive Bayes Classifier (Gaussian),0.6072,0.9185,0.6190,0.6072,0.5762,0.5485
4,Random Forest (Ensemble),0.8930,0.9884,0.8937,0.8930,0.8926,0.8747
5,XGBoost (Ensemble),0.9012,0.9900,0.9015,0.9012,0.9012,0.8841


#### **<span style="color:#8B4513"><u>Model Performance Visualization</u></span>**

In [12]:
# =============================================================================
# Visualization: Model Performance Comparison (Plotly - Interactive HTML)
# =============================================================================
metrics_to_plot = ['Accuracy', 'AUC_Score', 'Precision', 'Recall', 'F1_Score', 'MCC_Score']
model_names_list = val_comparison_df['Model'].tolist()

short_model_names = ['LR', 'DT', 'KNN', 'NB', 'RF', 'XGB']
model_name_mapping = dict(zip(model_names_list, short_model_names))

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'<b>{metric}</b>' for metric in metrics_to_plot],
    vertical_spacing=0.22,
    horizontal_spacing=0.12
)

colors = px.colors.qualitative.Set3[:len(model_names_list)]

for idx, metric in enumerate(metrics_to_plot):
    row = idx // 3 + 1
    col = idx % 3 + 1
    values = val_comparison_df[metric].values
    
    fig.add_trace(
        go.Bar(
            x=short_model_names,
            y=values,
            marker_color=colors,
            text=[f'{v:.3f}' for v in values],
            textposition='outside',
            textfont=dict(size=9),
            showlegend=False,
            hovertemplate='<b>%{customdata}</b><br>' + metric + ': %{y:.4f}<extra></extra>',
            customdata=model_names_list
        ),
        row=row, col=col
    )
    
    fig.update_yaxes(
        title=dict(text='Score', font=dict(size=8), standoff=3),
        range=[0, 1.18], 
        row=row, col=col
    )

    fig.update_xaxes(
        title=dict(text='Models', font=dict(size=8), standoff=3),
        tickfont=dict(size=10), 
        row=row, col=col
    )

fig.update_layout(
    title=dict(
        text='<b>Model Performance Comparison - Validation Metrics</b>',
        font=dict(size=18, color='white'),
        x=0.5
    ),
    height=900,
    template='plotly_dark',
    paper_bgcolor='#1e1e1e',
    plot_bgcolor='#2d2d2d',
    font=dict(color='white'),
    margin=dict(t=80, b=120, l=70, r=40)
)

legend_line1 = ' | '.join([f'{short}: {full}' for short, full in zip(short_model_names[:3], model_names_list[:3])])
legend_line2 = ' | '.join([f'{short}: {full}' for short, full in zip(short_model_names[3:], model_names_list[3:])])
fig.add_annotation(
    text=f"<b>Models:</b> {legend_line1}<br>{legend_line2}",
    xref="paper", yref="paper",
    x=0.5, y=-0.10,
    showarrow=False,
    font=dict(size=9, color='#cccccc'),
    align="center"
)

os.makedirs(REPORTS_DIR, exist_ok=True)
fig.write_html(f'{REPORTS_DIR}/{DATASET_NAME}_model_comparison_validation_metrics.html')
fig.show()
print(f"Interactive plot saved to '{REPORTS_DIR}/{DATASET_NAME}_model_comparison_validation_metrics.html'")

Interactive plot saved to '../artifacts/reports/kaggle_obesity_prediction_model_comparison_validation_metrics.html'


In [13]:
# =============================================================================
# Heatmap Visualization (Plotly - Interactive HTML)
# =============================================================================

fig_train_heatmap = go.Figure(data=go.Heatmap(
    z=train_comparison_df[metrics_to_plot].values,
    x=metrics_to_plot,
    y=short_model_names,
    text=[[f'{val:.4f}' for val in row] for row in train_comparison_df[metrics_to_plot].values],
    texttemplate='%{text}',
    textfont=dict(size=11),
    colorscale='RdYlGn',
    zmin=0, zmax=1,
    hovertemplate='Model: %{customdata}<br>Metric: %{x}<br>Score: %{z:.4f}<extra></extra>',
    customdata=[[name]*len(metrics_to_plot) for name in model_names_list]
))

fig_train_heatmap.update_layout(
    title=dict(text='<b>Training Metrics Heatmap</b>', font=dict(size=18, color='white'), x=0.5),
    height=550,
    template='plotly_dark',
    paper_bgcolor='#1e1e1e',
    plot_bgcolor='#2d2d2d',
    font=dict(color='white'),
    xaxis=dict(
        title=dict(text='Metrics', font=dict(size=10)),
        tickangle=0, 
        tickfont=dict(size=10)
    ),
    yaxis=dict(
        title=dict(text='Model', font=dict(size=10)),
        tickfont=dict(size=11)
    ),
    margin=dict(l=80, r=40, t=80, b=120)
)

fig_train_heatmap.add_annotation(
    text=f"<b>Models:</b> {legend_line1}<br>{legend_line2}",
    xref="paper", yref="paper",
    x=0.5, y=-0.18,
    showarrow=False,
    font=dict(size=9, color='#cccccc'),
    align="center"
)

fig_train_heatmap.write_html(f'{REPORTS_DIR}/{DATASET_NAME}_training_metrics_heatmap.html')
fig_train_heatmap.show()
print(f"Training heatmap saved to '{REPORTS_DIR}/{DATASET_NAME}_training_metrics_heatmap.html'")

Training heatmap saved to '../artifacts/reports/kaggle_obesity_prediction_training_metrics_heatmap.html'


In [14]:
# =============================================================================
# Validation Metrics Heatmap (Plotly - Interactive HTML)
# =============================================================================
fig_val_heatmap = go.Figure(data=go.Heatmap(
    z=val_comparison_df[metrics_to_plot].values,
    x=metrics_to_plot,
    y=short_model_names,
    text=[[f'{val:.4f}' for val in row] for row in val_comparison_df[metrics_to_plot].values],
    texttemplate='%{text}',
    textfont=dict(size=11),
    colorscale='RdYlGn',
    zmin=0, zmax=1,
    hovertemplate='Model: %{customdata}<br>Metric: %{x}<br>Score: %{z:.4f}<extra></extra>',
    customdata=[[name]*len(metrics_to_plot) for name in model_names_list]
))

fig_val_heatmap.update_layout(
    title=dict(text='<b>Validation Metrics Heatmap</b>', font=dict(size=18, color='white'), x=0.5),
    height=550,
    template='plotly_dark',
    paper_bgcolor='#1e1e1e',
    plot_bgcolor='#2d2d2d',
    font=dict(color='white'),
    xaxis=dict(
        title=dict(text='Metrics', font=dict(size=10)),
        tickangle=0, 
        tickfont=dict(size=10)
    ),
    yaxis=dict(
        title=dict(text='Model', font=dict(size=10)),
        tickfont=dict(size=11)
    ),
    margin=dict(l=80, r=40, t=80, b=120)
)

fig_val_heatmap.add_annotation(
    text=f"<b>Models:</b> {legend_line1}<br>{legend_line2}",
    xref="paper", yref="paper",
    x=0.5, y=-0.18,
    showarrow=False,
    font=dict(size=9, color='#cccccc'),
    align="center"
)

fig_val_heatmap.write_html(f'{REPORTS_DIR}/{DATASET_NAME}_validation_metrics_heatmap.html')
fig_val_heatmap.show()
print(f"Validation heatmap saved to '{REPORTS_DIR}/{DATASET_NAME}_validation_metrics_heatmap.html'")

Validation heatmap saved to '../artifacts/reports/kaggle_obesity_prediction_validation_metrics_heatmap.html'


In [15]:
# =============================================================================
# Confusion Matrices (Plotly - Interactive HTML)
# =============================================================================
short_class_names = ['IW', 'NW', 'NOR', 'OB-I', 'OB-II', 'OB-III', 'OW']
class_name_map = dict(zip(short_class_names, class_names))

class_legend_line1 = ' | '.join([f'{s}: {f}' for s, f in zip(short_class_names[:4], class_names[:4])])
class_legend_line2 = ' | '.join([f'{s}: {f}' for s, f in zip(short_class_names[4:], class_names[4:])])

fig_cm = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'<b>{short_model_names[i]}</b>' for i in range(len(results))],
    vertical_spacing=0.20,
    horizontal_spacing=0.14
)

for idx, (model_name, result) in enumerate(results.items()):
    row = idx // 3 + 1
    col = idx % 3 + 1
    cm = confusion_matrix(y_val, result['y_val_pred'])
    
    fig_cm.add_trace(
        go.Heatmap(
            z=cm,
            x=short_class_names,
            y=short_class_names,
            text=cm,
            texttemplate='%{text}',
            textfont=dict(size=8),
            colorscale='Blues',
            showscale=False,
            hovertemplate='Actual: %{customdata[0]}<br>Predicted: %{customdata[1]}<br>Count: %{z}<extra></extra>',
            customdata=[[[class_names[j], class_names[i]] for i in range(len(class_names))] for j in range(len(class_names))]
        ),
        row=row, col=col
    )
    
    fig_cm.update_xaxes(
        title=dict(text='Predicted', font=dict(size=8), standoff=3),
        tickangle=45, 
        tickfont=dict(size=7), 
        row=row, col=col
    )
    fig_cm.update_yaxes(
        title=dict(text='Actual', font=dict(size=8), standoff=3),
        tickfont=dict(size=7), 
        row=row, col=col
    )

fig_cm.update_layout(
    title=dict(
        text='<b>Confusion Matrices - Validation Set</b>',
        font=dict(size=18, color='white'),
        x=0.5
    ),
    height=950,
    template='plotly_dark',
    paper_bgcolor='#1e1e1e',
    plot_bgcolor='#2d2d2d',
    font=dict(color='white'),
    margin=dict(t=80, b=130, l=70, r=40)
)

fig_cm.add_annotation(
    text=f"<b>Classes:</b> {class_legend_line1}<br>{class_legend_line2}",
    xref="paper", yref="paper",
    x=0.5, y=-0.10,
    showarrow=False,
    font=dict(size=8, color='#cccccc'),
    align="center"
)

fig_cm.write_html(f'{REPORTS_DIR}/{DATASET_NAME}_confusion_matrices.html')
fig_cm.show()
print(f"Confusion matrices saved to '{REPORTS_DIR}/{DATASET_NAME}_confusion_matrices.html'")

Confusion matrices saved to '../artifacts/reports/kaggle_obesity_prediction_confusion_matrices.html'


In [16]:
# =============================================================================
# AUC-ROC Curves for All Models (One-vs-Rest Multi-class)
# =============================================================================
# Binarize the target for ROC curve calculation
y_val_bin = label_binarize(y_val, classes=list(range(num_classes)))

fig_roc = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'<b>{short_model_names[i]}</b>' for i in range(len(results))],
    vertical_spacing=0.22,
    horizontal_spacing=0.12
)

class_colors = px.colors.qualitative.Set2[:num_classes]

model_auc_values = {}

for idx, (model_name, result) in enumerate(results.items()):
    row = idx // 3 + 1
    col = idx % 3 + 1
    model_auc_values[model_name] = {}
    
    if result['y_val_proba'] is not None:
        y_proba = result['y_val_proba']
        
        # Calculate ROC curve for each class
        for i, cls_name in enumerate(class_names):
            fpr, tpr, _ = roc_curve(y_val_bin[:, i], y_proba[:, i])
            roc_auc = auc(fpr, tpr)
            model_auc_values[model_name][short_class_names[i]] = roc_auc
            
            fig_roc.add_trace(
                go.Scatter(
                    x=fpr, y=tpr,
                    mode='lines',
                    name=short_class_names[i],
                    line=dict(color=class_colors[i], width=1.5),
                    showlegend=False,
                    legendgroup=cls_name,
                    hovertemplate=f'{cls_name}<br>FPR: %{{x:.3f}}<br>TPR: %{{y:.3f}}<br>AUC: {roc_auc:.3f}<extra></extra>'
                ),
                row=row, col=col
            )
        
        fig_roc.add_trace(
            go.Scatter(
                x=[0, 1], y=[0, 1],
                mode='lines',
                line=dict(color='gray', dash='dash', width=1),
                showlegend=False,
                hoverinfo='skip'
            ),
            row=row, col=col
        )
    
    fig_roc.update_xaxes(
        title=dict(text='FPR', font=dict(size=8), standoff=2),
        range=[0, 1], 
        tickfont=dict(size=7),
        tickvals=[0, 0.5, 1],
        row=row, col=col
    )
    fig_roc.update_yaxes(
        title=dict(text='TPR', font=dict(size=8), standoff=2),
        range=[0, 1], 
        tickfont=dict(size=7),
        tickvals=[0, 0.5, 1],
        row=row, col=col
    )

# Calculate weighted average AUC for each model and add as annotation inside subplot
for idx, model_name in enumerate(results.keys()):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    if model_name in model_auc_values and model_auc_values[model_name]:
        avg_auc = np.mean(list(model_auc_values[model_name].values()))
        fig_roc.add_annotation(
            text=f'<b>Avg AUC: {avg_auc:.3f}</b>',
            xref=f'x{idx+1}' if idx > 0 else 'x',
            yref=f'y{idx+1}' if idx > 0 else 'y',
            x=0.95, y=0.05,
            showarrow=False,
            font=dict(size=8, color='#66ff66'),
            bgcolor='rgba(0,0,0,0.5)',
            borderpad=2,
            xanchor='right',
            yanchor='bottom'
        )

fig_roc.update_layout(
    title=dict(
        text='<b>ROC Curves - One-vs-Rest (Multi-class Classification)</b>',
        font=dict(size=18, color='white'),
        x=0.5
    ),
    height=950,
    template='plotly_dark',
    paper_bgcolor='#1e1e1e',
    plot_bgcolor='#2d2d2d',
    font=dict(color='white'),
    showlegend=False,
    margin=dict(t=80, b=120, l=70, r=40)
)

fig_roc.add_annotation(
    text=f"<b>Classes:</b> {class_legend_line1}<br>{class_legend_line2}",
    xref="paper", yref="paper",
    x=0.5, y=-0.08,
    showarrow=False,
    font=dict(size=8, color='#cccccc'),
    align="center"
)

fig_roc.write_html(f'{REPORTS_DIR}/{DATASET_NAME}_auc_roc_curves.html')
fig_roc.show()
print(f"AUC-ROC curves saved to '{REPORTS_DIR}/{DATASET_NAME}_auc_roc_curves.html'")

AUC-ROC curves saved to '../artifacts/reports/kaggle_obesity_prediction_auc_roc_curves.html'


#### **<span style="color:#8B4513"><u>Model Observations & Analysis</u></span>**

In [17]:
# =============================================================================
# Model Observations & Analysis Table
# =============================================================================
# Generate observations based on model performance characteristics

def generate_model_observations(val_df, train_df, results):
    """
    Generate detailed observations for each model based on performance metrics.
    """
    observations = []
    
    for model_name in val_df['Model']:
        val_row = val_df[val_df['Model'] == model_name].iloc[0]
        train_row = train_df[train_df['Model'] == model_name].iloc[0]
        
        # Calculate overfitting indicator (train - val difference)
        overfit_score = train_row['Accuracy'] - val_row['Accuracy']
        
        # Determine performance tier
        if val_row['F1_Score'] >= 0.90:
            performance = 'Excellent'
        elif val_row['F1_Score'] >= 0.80:
            performance = 'Good'
        elif val_row['F1_Score'] >= 0.70:
            performance = 'Moderate'
        else:
            performance = 'Needs Improvement'
        
        # Determine overfitting status
        if overfit_score > 0.10:
            overfit_status = 'High Overfitting'
        elif overfit_score > 0.05:
            overfit_status = 'Mild Overfitting'
        elif overfit_score < -0.02:
            overfit_status = 'Underfitting'
        else:
            overfit_status = 'Well Generalized'
        
        # Generate specific observations based on model type
        if 'Logistic Regression' in model_name:
            strengths = 'Fast training, interpretable coefficients, good baseline'
            weaknesses = 'Linear decision boundaries, may underfit complex patterns'
            use_case = 'When interpretability is important, quick baseline'
        elif 'Decision Tree' in model_name:
            strengths = 'Highly interpretable, handles non-linear relationships'
            weaknesses = 'Prone to overfitting, unstable to small data changes'
            use_case = 'When model interpretability and visualization needed'
        elif 'K-Nearest' in model_name:
            strengths = 'Simple, no training phase, works well with small datasets'
            weaknesses = 'Slow prediction, sensitive to feature scaling and noise'
            use_case = 'Small datasets, when similar samples cluster together'
        elif 'Naive Bayes' in model_name:
            strengths = 'Very fast, works with limited data, probabilistic output'
            weaknesses = 'Strong independence assumption, may not capture correlations'
            use_case = 'Quick baseline, text classification, real-time prediction'
        elif 'Random Forest' in model_name:
            strengths = 'Reduces overfitting vs single tree, feature importance'
            weaknesses = 'Slower than single models, less interpretable'
            use_case = 'General purpose, when accuracy matters more than speed'
        elif 'XGBoost' in model_name:
            strengths = 'State-of-art accuracy, handles missing values, regularization'
            weaknesses = 'Complex tuning, longer training time, less interpretable'
            use_case = 'Competitions, when maximum accuracy is the goal'
        else:
            strengths = 'N/A'
            weaknesses = 'N/A'
            use_case = 'N/A'
        
        observations.append({
            'Model': model_name,
            'Val F1': f"{val_row['F1_Score']:.4f}",
            'Val Accuracy': f"{val_row['Accuracy']:.4f}",
            'Performance': performance,
            'Generalization': overfit_status,
            'Overfit Gap': f"{overfit_score:.4f}",
            'Strengths': strengths,
            'Weaknesses': weaknesses,
            'Best Use Case': use_case
        })
    
    return pd.DataFrame(observations)

# Generate observations table
observations_df = generate_model_observations(val_comparison_df, train_comparison_df, results)

# Display as styled table
print("Model Observations & Analysis")
print("="*100)
display(observations_df[['Model', 'Val F1', 'Val Accuracy', 'Performance', 'Generalization', 'Overfit Gap']])

Model Observations & Analysis


,Model,Val F1,Val Accuracy,Performance,Generalization,Overfit Gap
0,Logistic Regression,0.8638,0.8644,Good,Well Generalized,0.0028
1,Decision Tree Classifier,0.8779,0.8776,Good,Well Generalized,0.0249
2,K-Nearest Neighbor Classifier,0.7731,0.7752,Moderate,High Overfitting,0.2248
3,Naive Bayes Classifier (Gaussian),0.5762,0.6072,Needs Improvement,Well Generalized,0.0087
4,Random Forest (Ensemble),0.8926,0.8930,Good,Mild Overfitting,0.0743
5,XGBoost (Ensemble),0.9012,0.9012,Excellent,Mild Overfitting,0.0723


In [18]:
# =============================================================================
# Interactive Observations Table (Plotly - HTML)
# =============================================================================

fig_obs = go.Figure(data=[go.Table(
    columnwidth=[180, 120, 250, 250, 200],
    header=dict(
        values=['<b>Model</b>', '<b>Performance</b>', '<b>Strengths</b>', '<b>Weaknesses</b>', '<b>Best Use Case</b>'],
        fill_color='#FF6B9D',
        font=dict(color='#1e1e1e', size=12, family='Arial Black'),
        align='center',
        height=40
    ),
    cells=dict(
        values=[
            observations_df['Model'],
            observations_df['Performance'],
            observations_df['Strengths'],
            observations_df['Weaknesses'],
            observations_df['Best Use Case']
        ],
        fill_color=[['#2d2d2d' if i % 2 == 0 else '#3a3a3a' for i in range(len(observations_df))]] * 5,
        align=['left', 'center', 'left', 'left', 'left'],
        font=dict(color='white', size=10, family='Arial'),
        height=50
    )
)])

fig_obs.update_layout(
    title=dict(
        text='<b>Model Observations - Strengths, Weaknesses & Use Cases</b>',
        font=dict(size=18, color='white'),
        x=0.5
    ),
    height=450,
    template='plotly_dark',
    paper_bgcolor='#1e1e1e',
    font=dict(color='white'),
    margin=dict(l=20, r=20, t=60, b=20)
)

fig_obs.write_html(f'{REPORTS_DIR}/{DATASET_NAME}_model_observations.html')
fig_obs.show()
print(f"Model observations saved to '{REPORTS_DIR}/{DATASET_NAME}_model_observations.html'")

Model observations saved to '../artifacts/reports/kaggle_obesity_prediction_model_observations.html'


#### **<span style="color:#8B4513"><u>Best Model Selection</u></span>**

In [19]:
# Identify the best model based on validation F1 Score
best_model_name = val_comparison_df.loc[val_comparison_df['F1_Score'].idxmax(), 'Model']
best_model_metrics = results[best_model_name]['val_metrics']

print("Best Model Selection Summary")
print("="*100)
print(f"\nBest Performing Model: {best_model_name}")
print(f"\nValidation Metrics for Best Model:")
for metric, value in best_model_metrics.items():
    if value is not None:
        print(f"   • {metric}: {value:.4f}")

# Rankings for each metric
print("\nModel Rankings by Metric (Validation Set):")
print("-"*100)
for metric in metrics_to_plot:
    ranking = val_comparison_df.sort_values(metric, ascending=False)['Model'].tolist()
    print(f"\n{metric}:")
    for i, model in enumerate(ranking, 1):
        score = val_comparison_df.loc[val_comparison_df['Model'] == model, metric].values[0]
        print(f"   {i}. {model}: {score:.4f}")

Best Model Selection Summary

Best Performing Model: XGBoost (Ensemble)

Validation Metrics for Best Model:
   • Accuracy: 0.9012
   • AUC_Score: 0.9900
   • Precision: 0.9015
   • Recall: 0.9012
   • F1_Score: 0.9012
   • MCC_Score: 0.8841

Model Rankings by Metric (Validation Set):
----------------------------------------------------------------------------------------------------

Accuracy:
   1. XGBoost (Ensemble): 0.9012
   2. Random Forest (Ensemble): 0.8930
   3. Decision Tree Classifier: 0.8776
   4. Logistic Regression: 0.8644
   5. K-Nearest Neighbor Classifier: 0.7752
   6. Naive Bayes Classifier (Gaussian): 0.6072

AUC_Score:
   1. XGBoost (Ensemble): 0.9900
   2. Random Forest (Ensemble): 0.9884
   3. Logistic Regression: 0.9818
   4. Decision Tree Classifier: 0.9749
   5. K-Nearest Neighbor Classifier: 0.9501
   6. Naive Bayes Classifier (Gaussian): 0.9185

Precision:
   1. XGBoost (Ensemble): 0.9015
   2. Random Forest (Ensemble): 0.8937
   3. Decision Tree Classifier: 0

In [20]:
# =============================================================================
# Save Evaluation Results
# =============================================================================

# Create directories if they don't exist
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

# Save results to JSON
model_evaluation_results = {
    'training_metrics': train_comparison_df.to_dict(orient='records'),
    'validation_metrics': val_comparison_df.to_dict(orient='records'),
    'model_observations': observations_df.to_dict(orient='records'),
    'best_model': {
        'name': best_model_name,
        'metrics': best_model_metrics
    },
    'class_names': class_names,
    'num_classes': num_classes
}

with open(f'{DATA_DIR}/{DATASET_NAME}_model_evaluation_results.json', 'w') as f:
    json.dump(model_evaluation_results, f, indent=2)

print(f"Model evaluation results saved to '{DATA_DIR}/{DATASET_NAME}_model_evaluation_results.json'")

train_comparison_df.to_csv(f'{DATA_DIR}/{DATASET_NAME}_training_metrics_comparison.csv', index=False)
val_comparison_df.to_csv(f'{DATA_DIR}/{DATASET_NAME}_validation_metrics_comparison.csv', index=False)
observations_df.to_csv(f'{DATA_DIR}/{DATASET_NAME}_model_observations.csv', index=False)

print(f"Training metrics saved to '{DATA_DIR}/{DATASET_NAME}_training_metrics_comparison.csv'")
print(f"Validation metrics saved to '{DATA_DIR}/{DATASET_NAME}_validation_metrics_comparison.csv'")
print(f"Model observations saved to '{DATA_DIR}/{DATASET_NAME}_model_observations.csv'")

Model evaluation results saved to '../artifacts/data/kaggle_obesity_prediction_model_evaluation_results.json'
Training metrics saved to '../artifacts/data/kaggle_obesity_prediction_training_metrics_comparison.csv'
Validation metrics saved to '../artifacts/data/kaggle_obesity_prediction_validation_metrics_comparison.csv'
Model observations saved to '../artifacts/data/kaggle_obesity_prediction_model_observations.csv'


---
### **<span style="color:#8B008B"><u>Save Pre-Trained Models</u></span>**

In [21]:
# =============================================================================
# Retrain ALL Models on Combined Data (Train + Validation) for Final Deployment
# =============================================================================

# Create models directory if it doesn't exist
os.makedirs(MODELS_DIR, exist_ok=True)

print("="*100)
print("RETRAINING ALL MODELS ON COMBINED DATA (TRAIN + VALIDATION)")
print("="*100)

# Combine training and validation data
X_combined = pd.concat([X_train, X_val], axis=0, ignore_index=True)
y_combined = pd.concat([y_train, y_val], axis=0, ignore_index=True)

print(f"\nCombined Dataset:")
print(f"   Training samples: {len(X_train)}")
print(f"   Validation samples: {len(X_val)}")
print(f"   Combined samples: {len(X_combined)}")
print(f"   Features: {X_combined.shape[1]}")

def create_model_instance(model_name, best_params):
    """Create a fresh model instance with the given tuned parameters."""
    if 'Logistic Regression' in model_name:
        return LogisticRegression(random_state=42, **best_params)
    elif 'Decision Tree' in model_name:
        return DecisionTreeClassifier(random_state=42, **best_params)
    elif 'K-Nearest' in model_name:
        return KNeighborsClassifier(**best_params)
    elif 'Naive Bayes' in model_name:
        return GaussianNB(**best_params)
    elif 'Random Forest' in model_name:
        return RandomForestClassifier(random_state=42, n_jobs=-1, **best_params)
    elif 'XGBoost' in model_name:
        return XGBClassifier(
            objective='multi:softprob',
            num_class=num_classes,
            random_state=42,
            eval_metric='mlogloss',
            n_jobs=-1,
            verbosity=0,
            **best_params
        )

# Retrain all models on combined data and save
final_models = {}

for model_name in results.keys():
    print(f"\n{'─'*80}")
    print(f"Retraining: {model_name}")
    
    best_params = tuning_results[model_name]['best_params']
    
    final_model = create_model_instance(model_name, best_params)
    
    final_model.fit(X_combined, y_combined)
    
    y_comb_pred = final_model.predict(X_combined)
    y_comb_proba = final_model.predict_proba(X_combined) if hasattr(final_model, 'predict_proba') else None
    comb_metrics = calculate_metrics(y_combined, y_comb_pred, y_comb_proba)
    
    clean_name = model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
    model_path = f'{MODELS_DIR}/{clean_name}.joblib'
    
    joblib.dump(final_model, model_path, compress=3)
    file_size = os.path.getsize(model_path) / 1024  # KB
    
    final_models[model_name] = {
        'model': final_model,
        'model_file': f'{clean_name}.joblib',
        'model_path': model_path,
        'tuned_parameters': {k: (v.item() if hasattr(v, 'item') else v) for k, v in best_params.items()},
        'combined_metrics': comb_metrics,
        'validation_metrics': results[model_name]['val_metrics'],
        'cv_score': tuning_results[model_name]['best_cv_score'],
        'cv_std': tuning_results[model_name]['cv_std'],
        'file_size_kb': round(file_size, 2)
    }
    
    print(f"   Combined Accuracy: {comb_metrics['Accuracy']:.4f}")
    print(f"   Saved: {model_path} ({file_size:.2f} KB)")

# =============================================================================
# Create JSON with ALL model details
# =============================================================================
print("\n" + "="*100)
print("Creating JSON with all model details...")
print("="*100)

# Identify best model
best_model_name = val_comparison_df.loc[val_comparison_df['F1_Score'].idxmax(), 'Model']

# Build model_trained.json
model_trained = {
    'dataset': {
        'name': DATASET_DISPLAY_NAME,
        'short_name': DATASET_NAME,
        'training_samples': len(X_train),
        'validation_samples': len(X_val),
        'combined_samples': len(X_combined),
        'num_features': X_combined.shape[1],
        'feature_names': list(X_combined.columns),
        'num_classes': num_classes,
        'class_names': class_names,
        'class_to_label': class_to_label,
        'label_to_class': label_to_class
    },
    'best_model': {
        'name': best_model_name,
        'model_file': final_models[best_model_name]['model_file'],
        'tuned_parameters': final_models[best_model_name]['tuned_parameters'],
        'validation_metrics': final_models[best_model_name]['validation_metrics'],
        'combined_metrics': final_models[best_model_name]['combined_metrics'],
        'cv_score': final_models[best_model_name]['cv_score'],
        'cv_std': final_models[best_model_name]['cv_std']
    },
    'all_models': {}
}

for model_name, info in final_models.items():
    model_trained['all_models'][model_name] = {
        'model_file': info['model_file'],
        'tuned_parameters': info['tuned_parameters'],
        'validation_metrics': info['validation_metrics'],
        'combined_metrics': info['combined_metrics'],
        'cv_score': info['cv_score'],
        'cv_std': info['cv_std'],
        'file_size_kb': info['file_size_kb']
    }

model_trained_path = f'{MODELS_DIR}/{DATASET_NAME}_model_trained.json'
with open(model_trained_path, 'w') as f:
    json.dump(model_trained, f, indent=2)

print(f"\n✓ {DATASET_NAME}_model_trained.json saved: {model_trained_path}")
print(f"   Best Model: {best_model_name}")
print(f"   Total Models Saved: {len(final_models)}")



print("\n" + "="*100)
print("All models retrained on combined data and saved successfully!")
print("="*100)

RETRAINING ALL MODELS ON COMBINED DATA (TRAIN + VALIDATION)

Combined Dataset:
   Training samples: 15879
   Validation samples: 2803
   Combined samples: 18682
   Features: 22

────────────────────────────────────────────────────────────────────────────────
Retraining: Logistic Regression
   Combined Accuracy: 0.8674
   Saved: ../models/logistic_regression.joblib (2.16 KB)

────────────────────────────────────────────────────────────────────────────────
Retraining: Decision Tree Classifier
   Combined Accuracy: 0.8999
   Saved: ../models/decision_tree_classifier.joblib (17.94 KB)

────────────────────────────────────────────────────────────────────────────────
Retraining: K-Nearest Neighbor Classifier
   Combined Accuracy: 1.0000
   Saved: ../models/k-nearest_neighbor_classifier.joblib (689.13 KB)

────────────────────────────────────────────────────────────────────────────────
Retraining: Naive Bayes Classifier (Gaussian)
   Combined Accuracy: 0.6146
   Saved: ../models/naive_bayes_c

---
### **<span style="color:#8B008B"><u>Test Data Evaluation</u></span>**

In [22]:
# =============================================================================
# Load and Preprocess Test Data
# =============================================================================

# Load test data
TEST_DATA_PATH = f'{DATA_DIR}/{DATASET_NAME}_test.csv'
test_df = pd.read_csv(TEST_DATA_PATH)

print("Test Data Loaded:")
print(f"   Shape: {test_df.shape}")
print(f"   Columns: {list(test_df.columns)}")
print(f"\nFirst few rows:")
display(test_df.head())

# Check if test data has target column (NObeyesdad)
has_labels = TARGET_COLUMN_ORIGINAL in test_df.columns
print(f"\nTest data has labels: {has_labels}")

Test Data Loaded:
   Shape: (2076, 18)
   Columns: ['id', 'Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad']

First few rows:


,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,6365,Female,26.000000,1.656504,111.884535,yes,yes,3.0,3.0,Sometimes,no,2.638896,no,0.000000,0.096576,Sometimes,Public_Transportation,Obesity_Type_III
1,11451,Male,18.000000,1.750000,52.000000,yes,yes,2.0,3.0,Frequently,no,2.000000,no,0.000000,2.000000,Sometimes,Public_Transportation,Insufficient_Weight
2,2066,Female,20.000000,1.600000,82.000000,yes,yes,2.0,1.0,Sometimes,no,2.000000,no,1.000000,0.000000,no,Public_Transportation,Obesity_Type_I
3,19776,Female,25.967627,1.756509,133.472641,yes,yes,3.0,3.0,Sometimes,no,2.856521,no,1.587406,0.625350,Sometimes,Public_Transportation,Obesity_Type_III
4,7594,Female,25.957740,1.624395,109.980145,yes,yes,3.0,3.0,Sometimes,no,2.320068,no,0.001015,0.472343,Sometimes,Public_Transportation,Obesity_Type_III



Test data has labels: True


In [23]:
# =============================================================================
# Preprocess Test Data
# =============================================================================
def preprocess_test_data(test_df, preprocessing_analysis):
    """
    Apply the same preprocessing transformations to test data as done during training.
    
    Steps:
    1. One-hot encode categorical columns
    2. Standard scale numerical columns using training mean/std
    3. Encode target variable (if present)
    """
    df = test_df.copy()
    
    # Get encoding and scaling parameters
    encoding_stats = preprocessing_analysis['preprocessing_stats']['encoding']
    scaling_params = preprocessing_analysis['scaling_params']
    target_encoding = preprocessing_analysis['preprocessing_stats']['target_encoding']
    
    # Store original id and target (if present)
    ids = df['id'].values
    has_target = TARGET_COLUMN_ORIGINAL in df.columns
    if has_target:
        target_values = df[TARGET_COLUMN_ORIGINAL].values
    
    # 1. One-Hot Encoding for categorical columns
    print("Applying One-Hot Encoding...")
    categorical_columns = list(encoding_stats.keys())
    
    for col in categorical_columns:
        if col in df.columns:
            new_columns = encoding_stats[col]['new_columns']
            
            # Create one-hot encoded columns
            for new_col in new_columns:
                category = new_col.replace(f"{col}_", "")
                df[new_col] = (df[col] == category).astype(int)
            
            # Drop original column
            df = df.drop(columns=[col])
    
    # 2. Drop target column if present (we'll encode it separately)
    if has_target and TARGET_COLUMN_ORIGINAL in df.columns:
        df = df.drop(columns=[TARGET_COLUMN_ORIGINAL])
    
    # 3. Standard Scaling using training parameters
    print("Applying Standard Scaling (using training mean/std)...")
    for col, params in scaling_params.items():
        if col in df.columns and col != 'id':
            mean = params['mean']
            std = params['std']
            df[col] = (df[col] - mean) / std
    
    # 4. Ensure columns are in the same order as training features
    expected_features = [f for f in preprocessing_analysis['feature_names'] if f != 'id']
    
    # Add missing columns with 0 values
    for col in expected_features:
        if col not in df.columns:
            df[col] = 0
    
    # Reorder columns to match training
    df = df[['id'] + expected_features]
    
    # 5. Encode target if present
    y_test = None
    if has_target:
        class_to_label = target_encoding['class_to_label']
        y_test = np.array([class_to_label[val] for val in target_values])
        print(f"Target encoded: {len(np.unique(y_test))} classes")
    
    print(f"Preprocessed test data shape: {df.shape}")
    
    return df, y_test

test_preprocessed_df, y_test = preprocess_test_data(test_df, preprocessing_analysis)

print("\nPreprocessed Test Data:")
print(f"   Shape: {test_preprocessed_df.shape}")
display(test_preprocessed_df.head())

Applying One-Hot Encoding...
Applying Standard Scaling (using training mean/std)...
Target encoded: 7 classes
Preprocessed test data shape: (2076, 23)

Preprocessed Test Data:
   Shape: (2076, 23)


,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Male,...,CAEC_Sometimes,CAEC_no,SMOKE_yes,SCC_yes,CALC_Sometimes,CALC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,6365,0.372767,-0.504977,0.907460,1.038467,0.338177,0.999289,-1.169709,-0.865633,-0.999937,...,0.431247,-0.113228,-0.108572,-0.184001,0.616200,-0.576308,-0.041271,-0.044937,0.498149,-0.151873
1,11451,-1.026860,0.565962,-1.360479,-0.842094,0.338177,-0.048872,-1.169709,2.293806,1.000063,...,-2.318859,-0.113228,-0.108572,-0.184001,0.616200,-0.576308,-0.041271,-0.044937,0.498149,-0.151873
2,2066,-0.676953,-1.152195,-0.224323,-0.842094,-2.500941,-0.048872,0.029659,-1.025936,-0.999937,...,0.431247,-0.113228,-0.108572,-0.184001,-1.622849,1.735183,-0.041271,-0.044937,0.498149,-0.151873
3,19776,0.367103,0.640518,1.725042,1.038467,0.338177,1.356321,0.734175,0.012064,-0.999937,...,0.431247,-0.113228,-0.108572,-0.184001,0.616200,-0.576308,-0.041271,-0.044937,0.498149,-0.151873
4,7594,0.365374,-0.872766,0.835337,1.038467,0.338177,0.476226,-1.168491,-0.241908,-0.999937,...,0.431247,-0.113228,-0.108572,-0.184001,0.616200,-0.576308,-0.041271,-0.044937,0.498149,-0.151873


In [24]:
# =============================================================================
# Evaluate ALL Models on Test Data
# =============================================================================

X_test = test_preprocessed_df.drop(columns=['id'])

print(f"Test Features Shape: {X_test.shape}")
print(f"Test Target Shape: {y_test.shape if y_test is not None else 'No labels'}")

test_results = {}

for model_name, info in final_models.items():
    loaded_model = joblib.load(info['model_path'])
    
    y_test_pred = loaded_model.predict(X_test)
    y_test_proba = loaded_model.predict_proba(X_test) if hasattr(loaded_model, 'predict_proba') else None
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    
    test_results[model_name] = {
        'model': loaded_model,
        'y_test_pred': y_test_pred,
        'y_test_proba': y_test_proba,
        'test_metrics': test_metrics
    }
    
    print(f"\n{model_name}:")
    for metric, value in test_metrics.items():
        if value is not None:
            print(f"   {metric}: {value:.4f}")

Test Features Shape: (2076, 22)
Test Target Shape: (2076,)

Logistic Regression:
   Accuracy: 0.8810
   AUC_Score: 0.9840
   Precision: 0.8805
   Recall: 0.8810
   F1_Score: 0.8806
   MCC_Score: 0.8605

Decision Tree Classifier:
   Accuracy: 0.8887
   AUC_Score: 0.9764
   Precision: 0.8890
   Recall: 0.8887
   F1_Score: 0.8886
   MCC_Score: 0.8696

K-Nearest Neighbor Classifier:
   Accuracy: 0.8059
   AUC_Score: 0.9573
   Precision: 0.8049
   Recall: 0.8059
   F1_Score: 0.8049
   MCC_Score: 0.7723

Naive Bayes Classifier (Gaussian):
   Accuracy: 0.6185
   AUC_Score: 0.9138
   Precision: 0.6174
   Recall: 0.6185
   F1_Score: 0.5880
   MCC_Score: 0.5607

Random Forest (Ensemble):
   Accuracy: 0.9041
   AUC_Score: 0.9905
   Precision: 0.9051
   Recall: 0.9041
   F1_Score: 0.9038
   MCC_Score: 0.8878

XGBoost (Ensemble):
   Accuracy: 0.9215
   AUC_Score: 0.9926
   Precision: 0.9218
   Recall: 0.9215
   F1_Score: 0.9214
   MCC_Score: 0.9080


In [25]:
# =============================================================================
# Test Set Confusion Matrices for ALL Models (Plotly - Interactive)
# =============================================================================
if y_test is not None:
    print("="*100)
    print("TEST SET EVALUATION RESULTS - ALL MODELS")
    print("="*100)
    
    best_test_pred = test_results[best_model_name]['y_test_pred']
    print(f"\nClassification Report - Best Model: {best_model_name}")
    print("="*100)
    print(classification_report(y_test, best_test_pred, target_names=class_names, digits=4))
    
    model_names_list = list(test_results.keys())
    
    fig_test_cm = make_subplots(
        rows=2, cols=3,
        subplot_titles=[f'<b>{short_model_names[i]}</b>' for i in range(len(test_results))],
        vertical_spacing=0.20,
        horizontal_spacing=0.14
    )
    
    for idx, (model_name, result) in enumerate(test_results.items()):
        row = idx // 3 + 1
        col = idx % 3 + 1
        cm = confusion_matrix(y_test, result['y_test_pred'])
        
        test_acc = result['test_metrics']['Accuracy']
        
        fig_test_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=short_class_names,
                y=short_class_names,
                text=cm,
                texttemplate='%{text}',
                textfont=dict(size=8),
                colorscale='Blues',
                showscale=False,
                hovertemplate='Actual: %{customdata[0]}<br>Predicted: %{customdata[1]}<br>Count: %{z}<extra></extra>',
                customdata=[[[class_names[j], class_names[i]] for i in range(len(class_names))] for j in range(len(class_names))]
            ),
            row=row, col=col
        )
        
        fig_test_cm.add_annotation(
            text=f'<b>Acc: {test_acc:.3f}</b>',
            xref=f'x{idx+1}' if idx > 0 else 'x',
            yref=f'y{idx+1}' if idx > 0 else 'y',
            x=len(short_class_names)-1, y=0,
            showarrow=False,
            font=dict(size=8, color='#66ff66'),
            bgcolor='rgba(0,0,0,0.6)',
            borderpad=2,
            xanchor='right',
            yanchor='bottom'
        )
        
        fig_test_cm.update_xaxes(
            title=dict(text='Predicted', font=dict(size=8), standoff=3),
            tickangle=45, tickfont=dict(size=7), row=row, col=col
        )
        fig_test_cm.update_yaxes(
            title=dict(text='Actual', font=dict(size=8), standoff=3),
            tickfont=dict(size=7), row=row, col=col
        )
    
    fig_test_cm.update_layout(
        title=dict(
            text='<b>Confusion Matrices - Test Set (All Models)</b>',
            font=dict(size=18, color='white'),
            x=0.5
        ),
        height=950,
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#2d2d2d',
        font=dict(color='white'),
        margin=dict(t=80, b=130, l=70, r=40)
    )
    
    fig_test_cm.add_annotation(
        text=f"<b>Classes:</b> {class_legend_line1}<br>{class_legend_line2}",
        xref="paper", yref="paper",
        x=0.5, y=-0.08,
        showarrow=False,
        font=dict(size=8, color='#cccccc'),
        align="center"
    )
    
    legend_line1 = ' | '.join([f'{short}: {full}' for short, full in zip(short_model_names[:3], model_names_list[:3])])
    legend_line2 = ' | '.join([f'{short}: {full}' for short, full in zip(short_model_names[3:], model_names_list[3:])])
    fig_test_cm.add_annotation(
        text=f"<b>Models:</b> {legend_line1}<br>{legend_line2}",
        xref="paper", yref="paper",
        x=0.5, y=-0.13,
        showarrow=False,
        font=dict(size=8, color='#cccccc'),
        align="center"
    )
    
    fig_test_cm.show()
    
    test_comparison = []
    for model_name, result in test_results.items():
        row = {'Model': model_name}
        row.update(result['test_metrics'])
        test_comparison.append(row)
    test_comparison_df = pd.DataFrame(test_comparison)
    
    print("\nTest Metrics Comparison (All Models):")
    print("="*100)
    display(test_comparison_df.style.format({
        'Accuracy': '{:.4f}',
        'AUC_Score': '{:.4f}',
        'Precision': '{:.4f}',
        'Recall': '{:.4f}',
        'F1_Score': '{:.4f}',
        'MCC_Score': '{:.4f}'
    }).highlight_max(subset=['Accuracy', 'AUC_Score', 'Precision', 'Recall', 'F1_Score', 'MCC_Score'], color='darkgreen'))
else:
    print("No test labels available - cannot calculate metrics")

TEST SET EVALUATION RESULTS - ALL MODELS

Classification Report - Best Model: XGBoost (Ensemble)
                     precision    recall  f1-score   support

Insufficient_Weight     0.9560    0.9484    0.9522       252
      Normal_Weight     0.8939    0.9026    0.8982       308
     Obesity_Type_I     0.8946    0.9038    0.8991       291
    Obesity_Type_II     0.9753    0.9723    0.9738       325
   Obesity_Type_III     1.0000    0.9975    0.9988       405
 Overweight_Level_I     0.8533    0.7901    0.8205       243
Overweight_Level_II     0.8246    0.8770    0.8500       252

           accuracy                         0.9215      2076
          macro avg     0.9140    0.9131    0.9132      2076
       weighted avg     0.9218    0.9215    0.9214      2076




Test Metrics Comparison (All Models):


,Model,Accuracy,AUC_Score,Precision,Recall,F1_Score,MCC_Score
0,Logistic Regression,0.8810,0.9840,0.8805,0.8810,0.8806,0.8605
1,Decision Tree Classifier,0.8887,0.9764,0.8890,0.8887,0.8886,0.8696
2,K-Nearest Neighbor Classifier,0.8059,0.9573,0.8049,0.8059,0.8049,0.7723
3,Naive Bayes Classifier (Gaussian),0.6185,0.9138,0.6174,0.6185,0.5880,0.5607
4,Random Forest (Ensemble),0.9041,0.9905,0.9051,0.9041,0.9038,0.8878
5,XGBoost (Ensemble),0.9215,0.9926,0.9218,0.9215,0.9214,0.9080


In [26]:
# =============================================================================
# Display Test Predictions (Best Model)
# =============================================================================

best_pred = test_results[best_model_name]['y_test_pred']

# Create submission dataframe with predictions
submission_df = pd.DataFrame({
    'id': test_preprocessed_df['id'].astype(int),
    'NObeyesdad_predicted': [label_to_class[str(p)] for p in best_pred]
})

# Add true labels if available
if y_test is not None:
    submission_df['NObeyesdad_actual'] = [label_to_class[str(t)] for t in y_test]
    submission_df['correct'] = submission_df['NObeyesdad_predicted'] == submission_df['NObeyesdad_actual']

print(f"Test Predictions - Best Model ({best_model_name}):")
display(submission_df.head(20))

Test Predictions - Best Model (XGBoost (Ensemble)):


,id,NObeyesdad_predicted,NObeyesdad_actual,correct
0,6365,Obesity_Type_III,Obesity_Type_III,True
1,11451,Insufficient_Weight,Insufficient_Weight,True
2,2066,Obesity_Type_I,Obesity_Type_I,True
3,19776,Obesity_Type_III,Obesity_Type_III,True
4,7594,Obesity_Type_III,Obesity_Type_III,True
5,5146,Obesity_Type_III,Obesity_Type_III,True
6,5581,Obesity_Type_III,Obesity_Type_III,True
7,14359,Obesity_Type_III,Obesity_Type_III,True
8,15566,Normal_Weight,Overweight_Level_I,False
9,3008,Obesity_Type_III,Obesity_Type_III,True


---
### **<span style="color:#8B008B"><u>Final Summary</u></span>**

In [27]:
# =============================================================================
# Final Complete Summary
# =============================================================================
print("="*100)
print("COMPLETE MODEL TRAINING, TUNING & EVALUATION SUMMARY")
print("="*100)

# Get test metrics for best model
best_test_metrics = test_results[best_model_name]['test_metrics']

print(f"""
Dataset: {DATASET_NAME}
{'='*100}

DATA SPLITS:
  - Training Samples: {len(X_train)}
  - Validation Samples: {len(X_val)}
  - Combined (Train+Val) for Final Models: {len(X_combined)}
  - Test Samples: {len(X_test)}
  - Number of Features: {X_train.shape[1]}
  - Number of Classes: {num_classes}
  - Class Names: {class_names}

MODELS TRAINED & TUNED (RandomizedSearchCV, 5-fold CV):
""")

for i, (model_name, result) in enumerate(tuning_results.items(), 1):
    print(f"  {i}. {model_name}")
    print(f"     CV Score: {result['best_cv_score']:.4f} (+/- {result['cv_std']:.4f})")

print(f"""
{'='*100}
BEST MODEL SELECTION:
  - Model: {best_model_name}
  - Best Tuned Parameters: {final_models[best_model_name]['tuned_parameters']}

PERFORMANCE COMPARISON (Best Model):
{'='*100}
                          Validation    Test
  Accuracy:               {results[best_model_name]['val_metrics']['Accuracy']:.4f}        {best_test_metrics['Accuracy']:.4f}
  AUC Score:              {results[best_model_name]['val_metrics']['AUC_Score']:.4f}        {best_test_metrics['AUC_Score']:.4f}
  Precision:              {results[best_model_name]['val_metrics']['Precision']:.4f}        {best_test_metrics['Precision']:.4f}
  Recall:                 {results[best_model_name]['val_metrics']['Recall']:.4f}        {best_test_metrics['Recall']:.4f}
  F1 Score:               {results[best_model_name]['val_metrics']['F1_Score']:.4f}        {best_test_metrics['F1_Score']:.4f}
  MCC Score:              {results[best_model_name]['val_metrics']['MCC_Score']:.4f}        {best_test_metrics['MCC_Score']:.4f}

{'='*100}
ARTIFACTS GENERATED:
{'='*100}

Models (All retrained on combined data) - {MODELS_DIR}/:""")

for model_name, info in final_models.items():
    print(f"  - {info['model_file']} ({info['file_size_kb']} KB)")
print(f"  - {DATASET_NAME}_model_trained.json (all model details + best model)")

print(f"""
Reports (Interactive HTML) - {REPORTS_DIR}/:
  - {DATASET_NAME}_model_comparison_validation_metrics.html
  - {DATASET_NAME}_training_metrics_heatmap.html
  - {DATASET_NAME}_validation_metrics_heatmap.html
  - {DATASET_NAME}_confusion_matrices.html
  - {DATASET_NAME}_auc_roc_curves.html
  - {DATASET_NAME}_model_observations.html

Data Files - {DATA_DIR}/:
  - {DATASET_NAME}_model_evaluation_results.json
  - {DATASET_NAME}_training_metrics_comparison.csv
  - {DATASET_NAME}_validation_metrics_comparison.csv
  - {DATASET_NAME}_model_observations.csv
""")

print("="*100)
print("MODEL TRAINING, TUNING & EVALUATION COMPLETE!")
print("="*100)

COMPLETE MODEL TRAINING, TUNING & EVALUATION SUMMARY

Dataset: kaggle_obesity_prediction

DATA SPLITS:
  - Training Samples: 15879
  - Validation Samples: 2803
  - Combined (Train+Val) for Final Models: 18682
  - Test Samples: 2076
  - Number of Features: 22
  - Number of Classes: 7
  - Class Names: ['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I', 'Overweight_Level_II']

MODELS TRAINED & TUNED (RandomizedSearchCV, 5-fold CV):

  1. Logistic Regression
     CV Score: 0.8640 (+/- 0.0018)
  2. Decision Tree Classifier
     CV Score: 0.8739 (+/- 0.0042)
  3. K-Nearest Neighbor Classifier
     CV Score: 0.7819 (+/- 0.0070)
  4. Naive Bayes Classifier (Gaussian)
     CV Score: 0.6097 (+/- 0.0127)
  5. Random Forest (Ensemble)
     CV Score: 0.8946 (+/- 0.0045)
  6. XGBoost (Ensemble)
     CV Score: 0.9062 (+/- 0.0014)

BEST MODEL SELECTION:
  - Model: XGBoost (Ensemble)
  - Best Tuned Parameters: {'colsample_bytree': 0.68